# Course's network data

In this notebook we will analyze the similarity between different courses. <br>
Each course corresponds to a set of students who were enrolled in it, we will define the 'similarity' between two courses as the Jaccard coefficient between the two set of students who were enrolled in those courses. <br>


---
<b> An example: </b> <br>
Take two courses: Machine learning and  Applied data analysis. <br>
We define the students enrolled in ML course as the set A, the ones enrolled in ADA as B. <br>
Then, the similarity between those two courses is equal to the Jaccard coefficient between A and B. <br>
<img src="../images/jaccard.png"> <br>

---
<br>
For our final visualization, we will build a network in which the courses will be nodes and, each course will be connected to the most 'similar' courses. <br>
In this notebook, we will build the underlying network. We will apply an heuristic that will create a planar graph in order to help us in a latter phase of our visualization.


## 1. Load the data set

First, we will import a set of libraries that will be useful for our analysis.

In [10]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import networkx as nx

%matplotlib inline

Now, we will red the data. They are divided into 3 main data sets:
* <b> Student: </b> contains informations about each student erolled at EPFL
* <b> Courses: </b> contains informations about all courses offere by EPFL at master level
* <b> Enrollment: </b> contains informations about student enrollment in courses

In [11]:
df_student = pd.read_csv('../../data/csv/student.csv')
df_course = pd.read_csv('../../data/csv/server/courses.csv')
df_enrollment = pd.read_csv('../../data/csv/server/enrollment.csv')

In [12]:
df_course.head(2)

,Unnamed: 0,course_id,course_name,year
0,0,0,Biological and physiological transport,2006-2007
1,1,1,Biological and physiological transport,2007-2008


In [13]:
df_enrollment.head()

,Unnamed: 0,student_id,course_id,semester
0,0,2959,0,Master semestre 2
1,1,3692,0,Master semestre 2
2,2,9206,0,Master semestre 2
3,3,10336,0,Master semestre 2
4,4,21467,0,Master semestre 2


In [14]:
df_student.head()

,Unnamed: 0,student_name,section,student_id
0,0,Aabid Fouad,Génie mécanique,0
1,1,Aamodt Simen,Génie mécanique,1
2,2,Aanhaanen Simone,Architecture,2
3,3,Aapro Laurent,Systèmes de communication - master,3
4,4,Aapro Niccolò,Programme Sciences humaines et sociales,4


## 2. Set of students enrolled in each course

For each course we will now compute the set of students attending that course, we will create a file 'jaccard.csv' to save this info as we will use it often in the visualization.

In [15]:
df_complete = df_course.merge(df_enrollment, on='course_id')
df_jaccard = df_complete.groupby('course_name')['student_id'].agg({'set': lambda x: set(x)}).reset_index()
df_jaccard.head(2)

/anaconda3/envs/ada/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)




,course_name,set
0,Numerical approximation of PDE's II,"{33792, 18439, 36362, 15374, 17428, 31252, 189..."
1,A Political History of Urban Form,"{26634, 44558, 2579, 9748, 39447, 43544, 38941..."


In [16]:
df_jaccard.to_csv('../../data/csv/jaccard.csv')

We will particularly use this file in order to compute the jaccard coefficient between two courses using the students enrolled over the years.

## 2. The course network

Now, we will compute the connection between the courses in our database. We will again use the jaccard coefficient to obtain a similarity measure between courses.

In [17]:
df_jaccard['key'] = 1
df_product = df_jaccard.merge(df_jaccard, on='key')
df_product.drop('key', axis=1, inplace=True)
df_product.head(2)

,course_name_x,set_x,course_name_y,set_y
0,Numerical approximation of PDE's II,"{33792, 18439, 36362, 15374, 17428, 31252, 189...",Numerical approximation of PDE's II,"{33792, 18439, 36362, 15374, 17428, 31252, 189..."
1,Numerical approximation of PDE's II,"{33792, 18439, 36362, 15374, 17428, 31252, 189...",A Political History of Urban Form,"{26634, 44558, 2579, 9748, 39447, 43544, 38941..."


In [18]:
def jaccard(s1, s2):
    common = 0
    for e in s1:
        if e in s2:
            common +=1
    return common / (len(s1) + len(s2) - common)

df_edges = df_product.copy()
df_edges['jaccard'] = df_product.apply(lambda x: jaccard(x['set_x'], x['set_y']), axis=1)
#df_edges['jaccard'] = df_save
df_edges.drop(['set_x', 'set_y'], axis=1, inplace=True)
df_edges.head()

,course_name_x,course_name_y,jaccard
0,Numerical approximation of PDE's II,Numerical approximation of PDE's II,1.0
1,Numerical approximation of PDE's II,A Political History of Urban Form,0.0
2,Numerical approximation of PDE's II,A guided tour for engineers in applied stochas...,0.0
3,Numerical approximation of PDE's II,A history of abstraction in architecture,0.0
4,Numerical approximation of PDE's II,A network tour of data science,0.0


Save all the jaccard coefficient computed.

In [19]:
df_edges.to_csv('../../data/csv/all_edges.csv')

Now, we will apply an euristic method in order to keep only the most relevant edges. For each node we will keep the top 5% of its neighbours (jaccard > 0).

In [20]:
df_count = df_edges[df_edges['jaccard'] > 0].groupby('course_name_x').count().drop('course_name_y', axis=1).reset_index()
df_count.rename(columns={"course_name_x": "course_name", "jaccard": "neighbours"}, inplace=True)
df_count.head()

,course_name,neighbours
0,Numerical approximation of PDE's II,235
1,A Political History of Urban Form,176
2,A guided tour for engineers in applied stochas...,185
3,A history of abstraction in architecture,154
4,A network tour of data science,625


In [21]:
size_map = {}
for index, row in df_count.iterrows():
    size_map[row['course_name']] = row['neighbours']
len(size_map) #1976

2109

In [22]:
# How many isolated nodes? Looks like none 
df_count[df_count['neighbours'] == 0]

,course_name,neighbours


In [23]:
# Remove self-loops
df_edges = df_edges[df_edges['course_name_x'] != df_edges['course_name_y']]
# Remove edges with zero value
df_edges = df_edges[df_edges['jaccard'] > 0]
df_edges.head(5)

,course_name_x,course_name_y,jaccard
13,Numerical approximation of PDE's II,Advanced algorithms,0.002639
28,Numerical approximation of PDE's II,Advanced computer graphics,0.001597
49,Numerical approximation of PDE's II,Advanced methods in computational solid mechanics,0.012048
50,Numerical approximation of PDE's II,Advanced multiprocessor architecture,0.005208
56,Numerical approximation of PDE's II,Advanced probability and applications,0.003425


## Version 0: too crowded

In [24]:
def keep_friends(g):
    for course_name in set(g['course_name_x']):
        return g.nlargest(5, "jaccard")

df_edges_planar = df_edges.groupby('course_name_x', group_keys=False).apply(keep_friends)
df_edges_planar.head(20)

,course_name_x,course_name_y,jaccard
1268,Numerical approximation of PDE's II,Numerical approximation of PDE's I,0.280220
1273,Numerical approximation of PDE's II,Numerical integration of dynamical systems,0.244898
361,Numerical approximation of PDE's II,Computational linear algebra,0.156028
1272,Numerical approximation of PDE's II,Numerical integration of stochastic different...,0.151961
1274,Numerical approximation of PDE's II,Numerical methods for conservation laws,0.144578
2589,A Political History of Urban Form,Difficult Double: Trouble with Classicists,0.277344
2539,A Political History of Urban Form,Critique du projet urbain contemporain,0.197581
4203,A Political History of Urban Form,Why care? Une introduction à l'art contemporain,0.185897
4196,A Political History of Urban Form,"Visionnaires éclectiques. Architectures, 2000-...",0.183673
2239,A Political History of Urban Form,Apparent realities - Constructing the view,0.158491


In [25]:
df_edges_planar.to_csv('../../data/csv/server/course_network_v0.csv')

## Version 1: planar but unconnected

Do not run, too long and we are not using this version! <br>
Go to <b>Version 2</b>

In [ ]:
len(df_edges_planar)

In [ ]:
2893*3

In [ ]:
# Sorting the edges: first the less important
df_edges_planar = df_edges_planar.sort_values('jaccard', ascending=True)
df_edges_planar.head()

Let's take a look at our graph. Is it connected?

In [ ]:
nx.is_connected(graph)

It is not connected, we can print the size of each connected component to have an idea of its topology.

In [ ]:
[len(c) for c in sorted(nx.connected_components(graph), key=len, reverse=True)]

Now, we will try to generate a planar graph.

In [ ]:
graph = nx.from_pandas_edgelist(df_edges_planar, source='course_name_x', target='course_name_y')
nx.algorithms.planarity.check_planarity(graph)

In [ ]:
# [len(c) if len(c) > 20 else 'sob' for c in sorted(nx.connected_components(graph), key=len, reverse=True)]
lst_edges = df_edges_planar.values.tolist()
lst_edges[:3]

We will now remove the less impotant edges following the criteria:
* Take the edge with the smallest value with has not been saved
* If the edge does not disconnect the graph, remove it. Othewise save it and restart
* Check if graph is planar, if not repeat

In [ ]:
# WARNING: long running time
index = 0
removed = 0
cc = len(sorted(nx.connected_components(graph), key=len, reverse=True))
while not nx.algorithms.planarity.check_planarity(graph)[0]:
    source = lst_edges[index][0]
    target = lst_edges[index][1]
    
    if graph.has_edge(source, target):
        graph.remove_edge(source, target)
        if len(sorted(nx.connected_components(graph), key=len, reverse=True)) != cc:
            graph.add_edge(source, target)
        else:
            removed += 1
    index += 1
    if index % 1000 == 0:   
        print(index, removed)
    
nx.algorithms.planarity.check_planarity(graph)

Now, how many edges are left in our network?

In [ ]:
2 * len(graph.edges) / len(graph.nodes)

The average degree is 2, which is quite low!

In [ ]:
nx.draw(graph)

So, we decided to try to readd those edges which do not break planarity.

In [ ]:
df_edges_planar = df_edges_planar.sort_values('jaccard', ascending=False)
incr_edge_list = df_edges_planar.values.tolist()
incr_edge_list[:3]

In [ ]:
# WARNING: long running time
index = 0
added = 0
for edge in incr_edge_list:
    source = edge[0]
    target = edge[1]
    if not graph.has_edge(source, target):
        graph.add_edge(source, target)
        if not nx.algorithms.planarity.check_planarity(graph)[0]:
            graph.remove_edge(source, target)
        else:
            added += 1
    index += 1
    if index % 1000 == 0:
        print(index, added)

nx.algorithms.planarity.check_planarity(graph)[0]
        

In [ ]:
df_edges_planar = nx.to_pandas_edgelist(graph)
df_edges_planar.head()

Save this edges in a CSV file.

In [ ]:
df_edges_planar.to_csv('../../data/csv/server/course_network_v1.csv')

## Version 2: better balance

In [26]:
def keep_3_friends(g):
    for course_name in set(g['course_name_x']):
        return g.nlargest(3, "jaccard")

df_edges_planar = df_edges.groupby('course_name_x', group_keys=False).apply(keep_3_friends)

In [27]:
df_edges_planar.to_csv('../../data/csv/server/course_network_v2.csv')